In [3]:
import pandas as pd

aislesDF = pd.read_csv('aisles.csv')
departmentDF = pd.read_csv('departments.csv')
Order_Products_Prior_DF = pd.read_csv('order_products__prior.csv')
Order_Products_Train_DF = pd.read_csv('order_products__train.csv')
ordersDF = pd.read_csv('orders.csv')
productsDF = pd.read_csv('products.csv')

In [4]:

Order_Product_Name_Prior = pd.merge(Order_Products_Prior_DF, 
                                    productsDF, how='left', on='product_id')


Prior_User_Order_Product = pd.merge(Order_Product_Name_Prior, 
                                    ordersDF, how='left', on='order_id')

In [5]:
# keep only those that same user_id exists in both train and prior
Prior_User_Order_Product_inner = pd.merge(Prior_User_Order_Product,
                                          ordersDF[ordersDF['eval_set']=='train'][['user_id','eval_set']], 
                                          how='inner', on='user_id')

In [6]:
Prior_User_Order_Product_inner.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,eval_set_x,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set_y
0,2,33120,1,1,Organic Egg Whites,86,16,202279,prior,3,5,9,8.0,train
1,2,28985,2,1,Michigan Organic Kale,83,4,202279,prior,3,5,9,8.0,train
2,2,9327,3,0,Garlic Powder,104,13,202279,prior,3,5,9,8.0,train
3,2,45918,4,1,Coconut Butter,19,13,202279,prior,3,5,9,8.0,train
4,2,30035,5,0,Natural Sweetener,17,13,202279,prior,3,5,9,8.0,train


In [7]:
# Mean of order_dow (order placed day of week)
# Mean of order_hour_of_day
# Mean of days_since_prior_order
# Total number of orders made
# Total number of products bought

user_habit = Prior_User_Order_Product_inner[['user_id','order_id','product_name','order_dow','order_hour_of_day','days_since_prior_order']]

In [10]:
import numpy as np

user_avg = user_habit.groupby('user_id')[['order_dow', 'order_hour_of_day', 'days_since_prior_order']].agg(np.nanmean)
# Total number of orders of each user
user_order = user_habit.groupby('user_id').order_id.nunique()
user_avg['num_of_orders'] = user_order

# Total number of products of each user
product_number = user_habit.groupby('user_id')['order_id'].agg('count')
user_avg['num_of_products'] = product_number

/var/folders/n9/m3glrnr55ml8zq0dkchhng440000gn/T/ipykernel_43095/2043567057.py:3: FutureWarning: The provided callable <function nanmean at 0x1078404c0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  user_avg = user_habit.groupby('user_id')[['order_dow', 'order_hour_of_day', 'days_since_prior_order']].agg(np.nanmean)


In [11]:
# add product name to each user
name_list = []
for p_name in user_habit.groupby('user_id')['product_name']:
        name_list.append(' '.join(p_name[1]))

# add the names to dataframe
user_avg['product_name'] = name_list    

In [12]:
# Take a glimpse of the dataset
user_avg.head()

#Prior_User_Order_Product_inner.to_csv("../output/Prior_User_Order_Product_inner.csv")
#user_avg.to_csv('../output/user_avg.csv')

,order_dow,order_hour_of_day,days_since_prior_order,num_of_orders,num_of_products,product_name
user_id,,,,,,
1,2.644068,10.542373,20.259259,10,59,Soda Original Beef Jerky Pistachios Organic St...
2,2.005128,10.441026,15.967033,14,195,Artichoke Spinach Dip Chipotle Beef & Pork Rea...
5,1.621622,15.729730,14.500000,4,37,Uncured Genoa Salami Plain Whole Milk Yogurt W...
7,1.728155,13.631068,13.546392,20,206,85% Lean Ground Beef Organic Apple Slices Appl...
8,4.204082,2.448980,30.000000,3,49,Organic Baby Spinach Michigan Organic Kale Bag...


In [19]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Word2Vec

# spark = SparkSession.builder.appName("User Habit").getOrCreate()


spark = SparkSession.builder \
    .appName("User Habit") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
# load data
# user_avg = pd.read_csv("user_avg.csv")
product_name_DF = pd.DataFrame(user_avg['product_name'])
# product_documentDF = spark.createDataFrame(product_name_DF)

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/shishantsharma/Library/Python/3.9/lib/python/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:
product_name_DF.head()

In [ ]:
# Sample a fraction of data due to large computation
sample_frac = 0.2
product_name_DF_sample  = product_name_DF.sample(frac = sample_frac, random_state=321)
sample_user_id  = product_name_DF_sample.index
print(sample_user_id)

In [ ]:
dataFrameList = []
for row in product_name_DF_sample['product_name']:
    tup = (row.split(' '),)
    dataFrameList.append(tup)

# Check by the length of output
print(len(dataFrameList))

In [14]:
# Input data of word2vec: Each row is a bag of words from a sentence or document.
N = len(dataFrameList)//100
mod = len(dataFrameList) % 100
documentDF = spark.createDataFrame(dataFrameList[0:100], ["product_name"])

for i in range(1,N):
    documentDF_sub = spark.createDataFrame(dataFrameList[100*i:100*(i+1)], ["product_name"])
    documentDF = documentDF.union(documentDF_sub)
    
documentDF_sub = spark.createDataFrame(dataFrameList[100*N:len(dataFrameList)], ["product_name"])
documentDF = documentDF.union(documentDF_sub)

NameError: name 'dataFrameList' is not defined

In [ ]:
# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=5, minCount=0, inputCol="product_name", outputCol="result")
model = word2Vec.fit(documentDF)

result = model.transform(documentDF)

In [ ]:
# Vectors: densevector
vectorized_features = [ ]
for row in result.collect():
    text, vector = row
    vectorized_features.append(vector)

In [ ]:
# Get values from densevector into array
vectorized_features_array=[]
for vectors in vectorized_features:
    vectorized_features_array.append(vectors.values)

In [ ]:
# Names of vectorized feature columns
col_names = []
for i in range(1,6):
    name = "vectorized_feature_" + str(i)
    col_names.append(name)
    
# A dataframe: each vectorized feature as one column
vectorized_features_DF = pd.DataFrame(np.array(vectorized_features_array).reshape(len(dataFrameList),5), 
                 columns = col_names)

# Add "user_id" column
vectorized_features_DF['user_id'] = sample_user_id

In [ ]:
# Take a glimpse
vectorized_features_DF.head()

Combine All Features: Concatenate word2vec feature with other features into one dataframe


In [ ]:
# slice user_avg with sampled user_id
user_avg_sampled = user_avg[user_avg.index.isin(sample_user_id)]
# set index as one column 'user_id'
user_avg_sampled.reset_index(level=0, inplace=True)
# merge two dfs on 'user_id'
user_habit_features = pd.merge(user_avg_sampled, vectorized_features_DF, how='inner', on="user_id")
# drop "product_name"
user_habit_features.drop('product_name', axis=1, inplace=True)

In [ ]:
# Take a glimpse
user_habit_features.head()